In [8]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm

In [ ]:
def RSXgboost(X_train, y_train, X_test, y_test, prob=False, metric='f1_macro'):
    params = {
            "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
            "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
            "min_child_weight" : [ 1, 3, 5, 7 ],
            "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
            "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    }

    classifier  =  xgb.XGBClassifier(verbosity = 0, silent=True)
    best_model = RandomizedSearchCV(classifier, param_distributions=params,
                                                                                n_iter=1, n_jobs = -1, 
                                                                                scoring=metric, cv=10, 
                                                                                verbose=0)
    
    best_model.fit(X_train, y_train, verbose=False)
    y_pred_prob = best_model.predict_proba(X_test)
    y_pred = best_model.predict(X_test)
    
    return y_pred, y_pred_prob, best_model

In [13]:
scores=[]

for k in tqdm(range(1,11)):
    
    train = pd.read_csv(f'../../k-folds/train- {k} .csv')
    test = pd.read_csv(f'../../k-folds/test- {k} .csv')
    
    x_train = train[['alt', 'temp2m', 'humidity', 'precip', 'atm', 'wind', 'm.fapar']].to_numpy()
    y_train = train['classes[cv.train, rotulo]'].to_numpy()
    
    x_test = test[['alt', 'temp2m', 'humidity', 'precip', 'atm', 'wind', 'm.fapar']].to_numpy()
    y_test = test['classes[cv.test, rotulo]'].to_numpy()
    
    params = {
            "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
            "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
            "min_child_weight" : [ 1, 3, 5, 7 ],
            "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
            "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    }
    

    classifier  =  xgb.XGBClassifier(verbosity = 0, silent=True)
    #best_model = RandomizedSearchCV(classifier, param_distributions=params,
    #                                                                            n_iter=2, n_jobs = -1, 
    #                                                                            scoring='accuracy', cv=10, 
    #                                                                            verbose=0)
    
    classifier.fit(x_train, y_train, verbose=False)
    y_pred = classifier.predict(x_test)

    scores.append(float(f'{cohen_kappa_score(y_test, y_pred):.2f}'))

print(scores)

100%|██████████| 10/10 [00:06<00:00,  1.50it/s]

[0.17, 0.24, 0.17, 0.36, 0.2, 0.29, 0.08, 0.17, 0.33, 0.17]
